# Exploring The Dimensions Search Language (DSL) - Deep Dive

This tutorial provides a detailed walkthrough of the most important features of the [Dimensions Search Language](https://docs.dimensions.ai/dsl/). 

This tutorial is based on the [Query Syntax](https://docs.dimensions.ai/dsl/language.html) section of the official documentation. So, it can be used as an interactive version of the documentation, as it allows to try out the various DSL queries presented there.

## What is the Dimensions Search Language?

The DSL aims to capture the type of interaction with Dimensions data
that users are accustomed to performing graphically via the [web
application](https://app.dimensions.ai/), and enable web app developers, power users, and others to
carry out such interactions by writing query statements in a syntax
loosely inspired by SQL but particularly suited to our specific domain
and data organization.

**Note:** this notebook uses the Python programming language, however all the **DSL queries are not Python-specific** and can in fact be reused with any other API client. 



## Prerequisites

This notebook assumes you have installed the [Dimcli](https://pypi.org/project/dimcli/) library and are familiar with the *Getting Started* tutorial.


In [1]:

# @markdown # Get the API library and login
# @markdown Click the 'play' button on the left (or shift+enter) after entering your API credentials

username = "" #@param {type: "string"}
password = "" #@param {type: "string"}
endpoint = "https://app.dimensions.ai" #@param {type: "string"}


!pip install dimcli plotly tqdm -U --quiet
import dimcli
from dimcli.shortcuts import *
dimcli.login(username, password, endpoint)
dsl = dimcli.Dsl()

#
# load common libraries
import time
import sys
import json
import pandas as pd
from pandas.io.json import json_normalize
from tqdm.notebook import tqdm as progress

#
# charts libs
# import plotly_express as px
import plotly.express as px
if not 'google.colab' in sys.modules:
  # make js dependecies local / needed by html exports
  from plotly.offline import init_notebook_mode
  init_notebook_mode(connected=True)

Dimcli - Dimensions API Client (v0.6.9)
Connected to endpoint: https://app.dimensions.ai - DSL version: 1.24
Method: dsl.ini file



## Sections Index 

1. Basic query structure
2. Full-text searching
3. Field searching
4. Searching for researchers
5. Returning results 
6. Aggregations

## 1. Basic query structure

DSL queries consist of two required components: a `search` phrase that
indicates the scientific records to be searched, and one or
more `return` phrases which specify the contents and structure of the
desired results.

The simplest valid DSL query is of the form `search <source>|return <result>`:

In [2]:
%%dsldf 
search grants return  grants limit 5

Returned Grants: 5 (total = 5310256)


title title_language  \
0  APPROACH to Enriching the Real World Evidence ...             en   
1  Molecular mechanism of DNA double strand break...             en   
2                     Life as concept and as science             en   
3  Scintillation Light For New Physics with Liqui...             en   
4  Jet quenching for heavy-ion collisions at the LHC             en   

   start_year                                            funders  start_date  \
0        2021  [{'id': 'grid.484521.e', 'country_name': 'Cana...  2021-11-30   
1        2021  [{'id': 'grid.424470.1', 'country_name': 'Belg...  2021-10-01   
2        2021  [{'id': 'grid.25111.36', 'country_name': 'Aust...  2021-10-01   
3        2021  [{'id': 'grid.270680.b', 'country_name': 'Belg...  2021-09-01   
4        2021  [{'id': 'grid.270680.b', 'country_name': 'Belg...  2021-09-01   

                           funding_org_name         active_year language  \
0  New Brunswick Health Research Foundation              [2021]       en   
1              Fund for Scientific Research              [2021]       en   
2                 FWF Austrian Science Fund  [2021, 2022, 2023]       en   
3                       European Commission  [2021, 2022, 2023]       en   
4                       European Commission  [2021, 2022, 2023]       en   

      project_num                                     original_title  \
0  2018-HRSI-1548  APPROACH to Enriching the Real World Evidence ...   
1        1301720F  Mécanismes moléculaires de la formation et la ...   
2          M 2734                     Life as concept and as science   
3          892933  Scintillation Light For New Physics with Liqui...   
4          893021  Jet quenching for heavy-ion collisions at the LHC   

              id    end_date  
0  grant.8690978         NaN  
1  grant.8950252         NaN  
2  grant.8715161  2023-09-30  
3  grant.8964235  2023-08-31  
4  grant.8963889  2023-08-31

### `search source`

A query must begin with the word `search` followed by a `source` name, i.e. the name of a type of scientific `record`, such as `grants` or `publications`.

**What are the sources available?** See the [data sources](https://docs.dimensions.ai/dsl/data-sources.html) section of the documentation. 

Alternatively, we can use the 'schema' API ([describe](https://docs.dimensions.ai/dsl/data-sources.html#metadata-api)) to return this information programmatically:

In [3]:
dsl.query("describe schema")

<dimcli.DslDataset object #4671900048. Dict keys: 'sources', 'entities'>

A more useful query might also make use of the optional `for` and
`where` phrases to limit the set of records returned.

In [4]:
%%dsldf 
search grants  for "lung cancer" 
    where active_year=2000 
return  grants  limit 5

Returned Grants: 5 (total = 1734)


id                                              title  \
0  grant.2386513   ROLE OF CD44 ISOFORMS IN ENDOTHELIAL CELL DAMAGE   
1  grant.2537116  ESTROGEN, ANGIOGENESIS AND ENDOTHELIAL PROGENI...   
2  grant.2537801  GENETIC ANALYSIS OF EPHRIN-EPH SIGNALING IN AN...   
3  grant.2536777  Synthetic Heparan Sulfate: Probing Biosynthesi...   
4  grant.2475193  Regulation of Telomerase by Sphingolipid Signa...   

     end_date title_language  project_num  \
0  2002-01-01             en  F32HL010455   
1  2004-11-30             en  R01HL063695   
2  2007-11-30             en  R01HL066221   
3  2017-12-31             en  R01HL062244   
4  2019-03-31             en  R01CA088932   

                                      original_title  \
0   ROLE OF CD44 ISOFORMS IN ENDOTHELIAL CELL DAMAGE   
1  ESTROGEN, ANGIOGENESIS AND ENDOTHELIAL PROGENI...   
2  GENETIC ANALYSIS OF EPHRIN-EPH SIGNALING IN AN...   
3  Synthetic Heparan Sulfate: Probing Biosynthesi...   
4  Regulation of Telomerase by Sphingolipid Signa...   

                          funding_org_name language  \
0  National Heart Lung and Blood Institute       en   
1  National Heart Lung and Blood Institute       en   
2  National Heart Lung and Blood Institute       en   
3  National Heart Lung and Blood Institute       en   
4                National Cancer Institute       en   

                                             funders  start_date  \
0  [{'id': 'grid.279885.9', 'linkout': ['http://w...  2000-12-31   
1  [{'id': 'grid.279885.9', 'linkout': ['http://w...  2000-12-18   
2  [{'id': 'grid.279885.9', 'linkout': ['http://w...  2000-12-18   
3  [{'id': 'grid.279885.9', 'linkout': ['http://w...  2000-12-15   
4  [{'id': 'grid.48336.3a', 'linkout': ['http://w...  2000-12-01   

                                         active_year  start_year  
0                                 [2000, 2001, 2002]        2000  
1                     [2000, 2001, 2002, 2003, 2004]        2000  
2   [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007]        2000  
3  [2000, 2001, 2002, 2003, 2004, 2005, 2006, 200...        2000  
4  [2000, 2001, 2002, 2003, 2004, 2005, 2006, 200...        2000

### `return` result (source or facet)

The most basic `return` phrase consists of the keyword `return` followed
by the name of a `record` or `facet` to be returned. 

This must be the
name of the `source` used in the `search` phrase, or the name of a
`facet` of that source.

In [5]:
%%dsldf
search grants for "laryngectomy" 
return grants limit 5

Returned Grants: 5 (total = 110)


id                                              title  \
0  grant.8554260  Wearable silent speech technology to enhance i...   
1  grant.8428997  Construction of a nursing system leading to im...   
2  grant.8441322  Development of self-directed TE shunt speech t...   
3  grant.8422934  Development of an olfactory improvement progra...   
4  grant.9013618  Early postoperative complications of laryngect...   

     end_date title_language  project_num  \
0  2024-07-31             en  R01DC016621   
1  2023-03-31             en     19H03937   
2  2022-03-31             en     19K10927   
3  2021-03-31             ja     19K19574   
4  2019-05-01             lv     AP-44/19   

                                      original_title  \
0  Wearable silent speech technology to enhance i...   
1  Construction of a nursing system leading to im...   
2  Development of self-directed TE shunt speech t...   
3           喉頭がん、下咽頭がんにより喉頭摘出術を受けた患者に対する嗅覚向上プログラムの開発   
4  Agrīnās laringektomiju pēcoperācijas komplikāc...   

                                    funding_org_name language  \
0  National Institute on Deafness and Other Commu...       en   
1         Japan Society for the Promotion of Science       ja   
2         Japan Society for the Promotion of Science       ja   
3         Japan Society for the Promotion of Science       ja   
4                  Ministry of Education and Science       lv   

                                             funders  start_date  \
0  [{'id': 'grid.214431.1', 'linkout': ['http://w...  2019-08-15   
1  [{'id': 'grid.54432.34', 'linkout': ['http://w...  2019-04-01   
2  [{'id': 'grid.54432.34', 'linkout': ['http://w...  2019-04-01   
3  [{'id': 'grid.54432.34', 'linkout': ['http://w...  2019-04-01   
4  [{'id': 'grid.453247.3', 'linkout': ['http://w...  2019-03-01   

                            active_year  start_year  
0  [2019, 2020, 2021, 2022, 2023, 2024]        2019  
1        [2019, 2020, 2021, 2022, 2023]        2019  
2              [2019, 2020, 2021, 2022]        2019  
3                    [2019, 2020, 2021]        2019  
4                                [2019]        2019

Eg let's see what are the *facets* available for the *grants* source:

In [6]:
fields = dsl.query("describe schema")['sources']['grants']['fields']
[x for x in fields if fields[x]['is_facet']]

['research_orgs',
 'category_bra',
 'active_year',
 'category_hrcs_hc',
 'research_org_state_codes',
 'category_uoa',
 'funding_currency',
 'category_for',
 'funding_org_acronym',
 'language',
 'category_icrp_cso',
 'category_icrp_ct',
 'funder_countries',
 'category_hrcs_rac',
 'category_rcdc',
 'start_year',
 'language_title',
 'research_org_cities',
 'research_org_countries',
 'category_hra',
 'researchers',
 'funding_org_name',
 'funders',
 'funding_org_city']

## 2. Full-text Searching

Full-text search or keyword search finds all instances of a term
(keyword) in a document, or group of documents. 

Full text search works
by using search indexes, which can be targeting specific sections of a
document e.g. its $abstract$, $authors$, $full text$ etc...

In [7]:
%%dsldf 
search publications 
    in full_data for "Apollo 11" 
return publications limit 5

Returned Errors: 1
1 QuerySyntaxError found
1 ParserError found
  * [Line 1:7] ('11') no viable alternative at input 'Apollo11'


### 2.1 `in [search index]`

This optional phrase consists of the particle `in` followed by a term indicating a `search index`, specifying for example whether the search
is limited to full text, title and abstract only, or title only. 

In [8]:
%%dsldf 
search grants 
    in title_abstract_only for "something" 
return grants limit 5

Returned Grants: 5 (total = 9677)


language                                     original_title  \
0       en  SaTC: CORE: Medium: Collaborative: Hardening O...   
1       en  SaTC: CORE: Medium: Collaborative: Hardening O...   
2       en  SaTC: CORE: Medium: Collaborative: Hardening O...   
3       en       The Cosmology of the Early and Late Universe   
4       en  Decoding the Infrared Spectra of High Frequenc...   

                                               title title_language  \
0  SaTC: CORE: Medium: Collaborative: Hardening O...             en   
1  SaTC: CORE: Medium: Collaborative: Hardening O...             en   
2  SaTC: CORE: Medium: Collaborative: Hardening O...             en   
3       The Cosmology of the Early and Late Universe             en   
4  Decoding the Infrared Spectra of High Frequenc...             en   

                                             funders  \
0  [{'id': 'grid.457785.c', 'latitude': 38.88058,...   
1  [{'id': 'grid.457785.c', 'latitude': 38.88058,...   
2  [{'id': 'grid.457785.c', 'latitude': 38.88058,...   
3  [{'id': 'grid.14467.30', 'latitude': 51.566532...   
4  [{'id': 'grid.457875.c', 'latitude': 38.880566...   

                      active_year  start_year  \
0  [2020, 2021, 2022, 2023, 2024]        2020   
1  [2020, 2021, 2022, 2023, 2024]        2020   
2  [2020, 2021, 2022, 2023, 2024]        2020   
3        [2020, 2021, 2022, 2023]        2020   
4        [2020, 2021, 2022, 2023]        2020   

                                    funding_org_name   project_num  \
0  Directorate for Computer & Information Science...       1954521   
1  Directorate for Computer & Information Science...       1955270   
2  Directorate for Computer & Information Science...       1954712   
3          Science and Technology Facilities Council  ST/T000732/1   
4   Directorate for Mathematical & Physical Sciences       1900095   

              id  start_date    end_date  
0  grant.9046367  2020-10-01  2024-09-30  
1  grant.9046432  2020-10-01  2024-09-30  
2  grant.9046384  2020-10-01  2024-09-30  
3  grant.8673892  2020-09-30  2023-09-29  
4  grant.8966252  2020-09-01  2023-08-31

Eg let's see what are the *search fields* available for the *grants* source:

In [9]:
dsl.query("describe schema")['sources']['grants']['search_fields']

['noun_phrases',
 'investigators',
 'full_data',
 'concepts',
 'title_abstract_only',
 'title_only']

In [10]:
%%dsldf 
search grants 
    in full_data for "graphene AND computer AND iron" 
return grants limit 5

Returned Grants: 5 (total = 10)


title    end_date  \
0  Weyl and Dirac semimetals and beyond - predict...  2021-12-31   
1  Project of the organization of the 18th Intern...  2018-12-31   
2  Subject subsidy for maintaining the research p...  2016-12-31   
3  Subject subsidy for maintaining the research p...  2015-12-31   
4  Intentional grant for conducting in 2014 the F...  2014-12-31   

  title_language  start_year  \
0             en        2019   
1             ru        2018   
2             pl        2016   
3             pl        2015   
4             pl        2014   

                                             funders  start_date  \
0  [{'id': 'grid.454869.2', 'country_name': 'Russ...  2019-01-01   
1  [{'id': 'grid.452899.b', 'country_name': 'Russ...  2018-01-01   
2  [{'id': 'grid.425823.a', 'country_name': 'Pola...  2016-02-22   
3  [{'id': 'grid.425823.a', 'country_name': 'Pola...  2015-02-19   
4  [{'id': 'grid.425823.a', 'country_name': 'Pola...  2014-04-09   

                           funding_org_name         active_year language  \
0                Russian Science Foundation  [2019, 2020, 2021]       en   
1     Russian Foundation for Basic Research              [2018]       ru   
2  Ministry of Science and Higher Education              [2016]       pl   
3  Ministry of Science and Higher Education              [2015]       pl   
4  Ministry of Science and Higher Education              [2014]       pl   

         project_num                                     original_title  \
0        19-43-04129  Weyl and Dirac semimetals and beyond - predict...   
1        18-02-20097  Проект организации 18-ой Международной конфере...   
2  4491/E-370/S/2016  Dotacja podmiotowa na utrzymanie potencjału ba...   
3  4491/E-370/S/2015  Dotacja podmiotowa na utrzymanie potencjału ba...   
4  4491/E-370/M/2014  Dotacja celowa na prowadzenie w 2014 przez Wyd...   

              id  
0  grant.8413990  
1  grant.8731867  
2  grant.7397800  
3  grant.7397795  
4  grant.7397490

Special search indexes for persons names permit to perform full text
searches on publications `authors` or grants `investigators`. Please see the
*Researchers Search* section below for more information
on how searches work in this case.

In [11]:
%dsldf search publications in authors for "\"Jennifer A Doudna\"" return publications limit 5

Returned Publications: 5 (total = 320)


title  \
0  Machine learning predicts new anti-CRISPR prot...   
1  Huge and variable diversity of episymbiotic CP...   
2  Cancer-specific loss of TERT activation sensit...   
3      Blueprint for a Pop-up SARS-CoV-2 Testing Lab   
4  Potent CRISPR-Cas9 inhibitors from Staphylococ...   

                                 author_affiliations volume issue  \
0  [[{'first_name': 'Simon', 'last_name': 'Eitzin...     48     9   
1  [[{'first_name': 'Christine Y', 'last_name': '...    NaN   NaN   
2  [[{'first_name': 'Alexandra M', 'last_name': '...    NaN   NaN   
3  [[{'first_name': 'Innovative Genomics Institut...    NaN   NaN   
4  [[{'first_name': 'Kyle E.', 'last_name': 'Watt...    117    12   

                 pages      type  year              id    journal.id  \
0            4698-4708   article  2020  pub.1125959258  jour.1018982   
1    2020.05.14.094862  preprint  2020  pub.1127645424  jour.1293558   
2    2020.04.25.061606  preprint  2020  pub.1127163455  jour.1293558   
3  2020.04.11.20061424   article  2020  pub.1126635310  jour.1369542   
4            6531-6539   article  2020  pub.1125548532  jour.1082971   

                                       journal.title  
0                             Nucleic Acids Research  
1                                            bioRxiv  
2                                            bioRxiv  
3                                            medRxiv  
4  Proceedings of the National Academy of Science...

### 2.2 `for "search term"`

This optional phrase consists of the keyword `for` followed by a
`search term` `string`, enclosed in double quotes (`"`).

Strings in double quotes can contain nested quotes escaped by a
backslash `\`. This will ensure that the string in nested double quotes
is searched for as if it was a single phrase, not multiple words.

An example of a phrase: `"\"Machine Learning\""` : results must contain
`Machine Learning` as a phrase.

In [12]:
%dsldf search publications for "\"Machine Learning\"" return publications limit 5

Returned Publications: 5 (total = 1131497)


title  \
0  Towards maritime traffic coordination in the e...   
1  Die Erfassung und Messung von Bedeutungsstrukt...   
2  Recognizing hotspots in Brief Eclectic Psychot...   
3  Capacitated vehicle routing problem with colum...   
4  Korean Technical Innovation: toward Autonomous...   

                                 author_affiliations    pages     type  year  \
0  [[{'first_name': 'Eetu', 'last_name': 'Heikkil...  243-248  chapter  2020   
1  [[{'first_name': 'Jan', 'last_name': 'Goldenst...  219-250  chapter  2020   
2  [[{'first_name': 'Sytske', 'last_name': 'Wiege...  1726672  article  2020   
3  [[{'first_name': 'Baze University Abuja', 'las...    41-54  article  2020   
4  [[{'first_name': 'Yongwon', 'last_name': 'Kwon...    83-94  chapter  2020   

               id volume issue    journal.id  \
0  pub.1124666091    NaN   NaN           NaN   
1  pub.1124034443    NaN   NaN           NaN   
2  pub.1125710665     11     1  jour.1045059   
3  pub.1126735888      3     1  jour.1365688   
4  pub.1124677880    NaN   NaN           NaN   

                                  journal.title  
0                                           NaN  
1                                           NaN  
2        European Journal of Psychotraumatology  
3  Open Journal of Discrete Applied Mathematics  
4                                           NaN

Example of multiple keywords: `"Machine Learning"` : this searches for
keywords independently.

In [13]:
%dsldf search publications for "Machine Learning" return publications limit 5

Returned Publications: 5 (total = 2388786)


id                                              title     type  \
0  pub.1125380179                       I. THE PHILOSOPHY OF SUCCESS  chapter   
1  pub.1125150382  3. Rural-Urban Divides and Digital Literacy in...  chapter   
2  pub.1124943680  4. Loss of the Social, Return of the Private: ...  chapter   
3  pub.1125152079                                               1839  chapter   
4  pub.1124947017  4. Visualizing the Division of Labor: William ...  chapter   

   year    pages                                author_affiliations  
0  2020   87-139                                                NaN  
1  2020    44-58  [[{'first_name': 'Daariimaa', 'last_name': 'Ma...  
2  2020    49-62  [[{'first_name': 'Mario', 'last_name': 'Hibert...  
3  2020  151-212                                                NaN  
4  2020   84-118  [[{'first_name': 'John', 'last_name': 'Barrell...

Note: Special characters, such as any of `^ " : ~ \ [ ] { } ( ) ! | & +` must be escaped by a backslash `\`. Also, please note escaping rules in
[Python](http://python-reference.readthedocs.io/en/latest/docs/str/escapes.html) (or other languages). For example, when writing a query with escaped quotes, such as `search publications for "\"phrase 1\" AND \"phrase 2\""`, in Python, it is necessary to escape the backslashes as well, so it
would look like: `'search publications for "\\"phrase 1\\" AND \\"phrase 2\\""'`. 

See the [official docs](https://docs.dimensions.ai/dsl/language.html#for-search-term) for more details. 




### 2.3 Boolean Operators

Search term can consist of multiple keywords or phrases connected using
boolean logic operators, e.g. `AND`, `OR` and `NOT`.

In [14]:
%dsldf search publications for "(dose AND concentration)" return publications limit 5

Returned Publications: 5 (total = 5252672)


title     pages     type  year  \
0                                     ANHANG. Part 2  802-1094  chapter  2020   
1                        England in 1845 and in 1885     61-66  chapter  2020   
2  Translational studies of estradiol and progest...   1723857  article  2020   
3  7. Conservation of the Amsterdam Sunflowers: F...   175-206  chapter  2020   
4  New findings questioning the construct validit...   1708145  article  2020   

               id                                author_affiliations volume  \
0  pub.1126070644                                                NaN    NaN   
1  pub.1126070808                                                NaN    NaN   
2  pub.1124948447  [[{'first_name': 'Antonia V', 'last_name': 'Se...     11   
3  pub.1125801745  [[{'first_name': 'Ella', 'last_name': 'Hendrik...    NaN   
4  pub.1124216519  [[{'first_name': 'Julian D', 'last_name': 'For...     11   

  issue    journal.id                           journal.title  
0   NaN           NaN                                     NaN  
1   NaN           NaN                                     NaN  
2     1  jour.1045059  European Journal of Psychotraumatology  
3   NaN           NaN                                     NaN  
4     1  jour.1045059  European Journal of Psychotraumatology

When specifying Boolean operators with keywords such as `AND`, `OR` and
`NOT`, the keywords must appear in all uppercase. 

The operators available are shown in the table below.
.

| Boolean Operator | Alternative Symbol | Description                                                                                                                                                                 |
|------------------|--------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `AND`            | `&&`               | Requires both terms on either side of the Boolean operator to be present for a match.                                                                                       |
| `NOT`            | `!`                | Requires that the following term not be present.                                                                                                                            |
| `OR`             | `||`               | Requires that either term (or both terms) be present for a match.                                                                                                           |
|                  | `+`                | Requires that the following term be present.                                                                                                                                |
|                  | `-`                | Prohibits the following term (that is, matches on fields or documents that do not include that term). The `-` operator is functionally similar to the Boolean operator `!`. |

In [15]:
%dsldf search publications for "(dose OR concentration) AND (-malaria +africa)" return publications limit 5

Returned Publications: 5 (total = 1351045)


title  \
0  Introduction: Murra, Materialism, Anthropology...   
1                  16. The Neuroethology of Birdsong   
2  17. Institutions for Infrastructure in Develop...   
3                  8. India in the Early Nuclear Age   
4                                               1864   

                                 author_affiliations    pages     type  year  \
0  [[{'first_name': 'John S.', 'last_name': 'Hend...      1-8  chapter  2020   
1  [[{'first_name': 'Eliot A.', 'last_name': 'Bre...  285-304  chapter  2020   
2  [[{'first_name': 'Antonio', 'last_name': 'Esta...  634-688  chapter  2020   
3  [[{'first_name': 'Srinath', 'last_name': 'Ragh...  129-143  chapter  2020   
4                                                NaN     3-53  chapter  2020   

               id  
0  pub.1125788851  
1  pub.1124946791  
2  pub.1124248682  
3  pub.1124248733  
4  pub.1126069081

### 2.4 Wildcard Searches

The DSL supports single and multiple character wildcard searches within
single terms. Wildcard characters can be applied to single terms, but
not to search phrases.

In [16]:
%dsldf search publications in title_only for "ital? malaria" return publications limit 5

Returned Publications: 5 (total = 142)


title  \
0  Seasons in Italy: Northern European travelers,...   
1  Updated guidelines for malaria prophylaxis in ...   
2  Clinical management of imported malaria in Ita...   
3  Investigation on potential malaria vectors (An...   
4  Increasing imported malaria in children and ad...   

                                 author_affiliations   pages     type  year  \
0  [[{'first_name': 'Benjamin', 'last_name': 'Rei...    1-20  article  2020   
1  [[{'first_name': 'Guido', 'last_name': 'Caller...  101544  article  2020   
2  [[{'first_name': 'Luciana', 'last_name': 'Lepo...   28-33  article  2020   
3  [[{'first_name': 'Valentina', 'last_name': 'Ta...     151  article  2019   
4  [[{'first_name': 'Fiorenza', 'last_name': 'Pan...   34-39  article  2019   

               id    journal.id                           journal.title  \
0  pub.1124231018  jour.1141817  Journal of Tourism and Cultural Change   
1  pub.1123222257  jour.1034401  Travel Medicine and Infectious Disease   
2  pub.1125332077  jour.1089291                          Microbiologica   
3  pub.1113815431  jour.1030597                         Malaria Journal   
4  pub.1113201846  jour.1034401  Travel Medicine and Infectious Disease   

  volume issue  
0    NaN   NaN  
1     33   NaN  
2     43     1  
3     18     1  
4     29   NaN

In [17]:
%dsldf search publications in title_only for "it* malaria" return publications limit 5

Returned Publications: 5 (total = 1493)


title  \
0  The effectiveness of older insecticide-treated...   
1  Modeling pyrethroids repellency and its role o...   
2  Severe-malaria infection and its outcomes amon...   
3  Variation of preventive practices and its asso...   
4  Malaria therapy in Spain: 100 years after its ...   

                                 author_affiliations volume issue  \
0  [[{'first_name': 'Monica P.', 'last_name': 'Sh...     19     1   
1  [[{'first_name': 'Berge', 'last_name': 'Tsanou...    136   NaN   
2  [[{'first_name': 'Toussaint', 'last_name': 'Ro...     33   NaN   
3  [[{'first_name': 'Mara', 'last_name': 'Ipa', '...     15     5   
4  [[{'first_name': 'Olga', 'last_name': 'Villasa...    NaN   NaN   

             pages     type  year              id    journal.id  \
0               24  article  2020  pub.1124106064  jour.1030597   
1           109809  article  2020  pub.1126819455  jour.1026215   
2           100333  article  2020  pub.1124902730  jour.1042240   
3         e0232909  article  2020  pub.1127450739  jour.1037553   
4  957154x20918824  article  2020  pub.1127262586  jour.1100702   

                              journal.title  
0                           Malaria Journal  
1                 Chaos Solitons & Fractals  
2  Spatial and Spatio-temporal Epidemiology  
3                                  PLoS ONE  
4                     History of Psychiatry

| Wildcard Search Type                                             | Special Character | Example                                                                                                                                                                                                                         |
|------------------------------------------------------------------|-------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Single character - matches a single character                    | `?`               | The search string `te?t` would match both `test` and `text`.                                                                                                                                                                    |
| Multiple characters - matches zero or more sequential characters | `*`               | The wildcard search: `tes*` would match `test`, `testing`, and `tester`. You can also use wildcard characters in the middle of a term. For example: `te*t` would match `test` and `text`. `*est` would match `pest` and `test`. |

### 2.5 Proximity Searches

A proximity search looks for terms that are within a specific distance
from one another.

To perform a proximity search, add the tilde character `~` and a numeric
value to the end of a search phrase. For example, to search for a
`formal` and `model` within 10 words of each other in a document, use
the search:

In [18]:
%dsldf search publications for "\"formal model\"~10" return publications limit 5

Returned Publications: 5 (total = 468213)


id                                              title  \
0  pub.1124248667  2. Clientelistic Politics and Economic Develop...   
1  pub.1125320181  Building cooperative learning to address alcoh...   
2  pub.1125144025                                           Foreword   
3  pub.1125788857      6. Hierarchy and Power in the Tropical Forest   
4  pub.1125789336  6. The Structure and Workings of Employer-Prom...   

                                 author_affiliations     type  year    pages  \
0  [[{'first_name': 'Pranab', 'last_name': 'Bardh...  chapter  2020   84-102   
1  [[{'first_name': 'Oladapo', 'last_name': 'Olad...  article  2020  1726722   
2                                                NaN  chapter  2020   xi-xvi   
3  [[{'first_name': 'Irving', 'last_name': 'Goldm...  chapter  2020  137-159   
4  [[{'first_name': 'Joseph F.', 'last_name': 'Ge...  chapter  2020  136-161   

  volume issue    journal.id         journal.title  
0    NaN   NaN           NaN                   NaN  
1     13     1  jour.1041075  Global Health Action  
2    NaN   NaN           NaN                   NaN  
3    NaN   NaN           NaN                   NaN  
4    NaN   NaN           NaN                   NaN

In [19]:
%dsldf search publications for "\"digital humanities\"~5  +ontology" return publications limit 5

Returned Publications: 5 (total = 7226)


title  \
0  Citizen science in the social sciences and hum...   
1  Emerging Extended Reality Technologies For Ind...   
2  WebKey: a graph-based method for event detecti...   
3  Describing, comparing and analysing digital ur...   
4  Social media as taste re-mediators: emerging p...   

                                 author_affiliations volume issue    pages  \
0  [[{'first_name': 'Loreta', 'last_name': 'Taugi...      6     1       89   
1  [[{'first_name': 'Jolanda G.', 'last_name': 'T...    NaN   NaN      NaN   
2  [[{'first_name': 'Elham', 'last_name': 'Rasoul...     54     3  585-604   
3  [[{'first_name': 'Vincent', 'last_name': 'Jail...     17   NaN   e00135   
4  [[{'first_name': 'Maria Giovanna', 'last_name'...     23     3     1-19   

        type  year              id    journal.id  \
0    article  2020  pub.1127423858  jour.1136613   
1  monograph  2020  pub.1127498852           NaN   
2    article  2020  pub.1120871378  jour.1327483   
3    article  2020  pub.1124427860  jour.1151279   
4    article  2020  pub.1124435145  jour.1327532   

                                       journal.title  
0                            Palgrave Communications  
1                                                NaN  
2         Journal of Intelligent Information Systems  
3  Digital Applications in Archaeology and Cultur...  
4                             Food Culture & Society

The distance referred to here is the number of term movements needed to match the specified phrase.  
In the example above, if `formal` and `model` were 10 spaces apart in a
field, but `formal` appeared before `model`, more than 10 term movements
would be required to move the terms together and position `formal` to
the right of `model` with a space in between.

## 3. Field Searching

Field searching allows to use a specific `field` of a `source` as a
query filter. For example, this can be a
[Literal](supported-types.ipynb) field such as the $type$ of a
publication, its $date$, $mesh terms$, etc.. Or it can be an
[entity](data-entities.ipynb) field, such as the $journal title$ for a
publication, the $country name$ of its author affiliations, etc..

**What are the fields available for each source?** See the [data sources](https://docs.dimensions.ai/dsl/data-sources.html) section of the documentation. 

Alternatively, we can use the 'schema' API ([describe](https://docs.dimensions.ai/dsl/data-sources.html#metadata-api)) to return this information programmatically: 

In [20]:
%dsldocs publications  

sources                       field           type  \
0   publications                   altmetric          float   
1   publications                altmetric_id        integer   
2   publications                     authors           json   
3   publications                    book_doi         string   
4   publications           book_series_title         string   
5   publications                  book_title         string   
6   publications                category_bra     categories   
7   publications                category_for     categories   
8   publications                category_hra     categories   
9   publications            category_hrcs_hc     categories   
10  publications           category_hrcs_rac     categories   
11  publications           category_icrp_cso     categories   
12  publications            category_icrp_ct     categories   
13  publications               category_rcdc     categories   
14  publications                category_sdg     categories   
15  publications                category_uoa     categories   
16  publications                    concepts         string   
17  publications                        date           date   
18  publications               date_inserted           date   
19  publications                         doi         string   
20  publications        field_citation_ratio          float   
21  publications            funder_countries      countries   
22  publications                     funders  organizations   
23  publications                          id         string   
24  publications                        issn         string   
25  publications                       issue         string   
26  publications                     journal       journals   
27  publications               journal_lists         string   
28  publications                     linkout         string   
29  publications                  mesh_terms         string   
30  publications      open_access_categories    open_access   
31  publications                       pages         string   
32  publications                       pmcid         string   
33  publications                        pmid         string   
34  publications           proceedings_title         string   
35  publications                   publisher         string   
36  publications            recent_citations        integer   
37  publications               reference_ids         string   
38  publications     relative_citation_ratio          float   
39  publications         research_org_cities         cities   
40  publications      research_org_countries      countries   
41  publications  research_org_country_names         string   
42  publications          research_org_names         string   
43  publications    research_org_state_codes         states   
44  publications    research_org_state_names         string   
45  publications               research_orgs  organizations   
46  publications                 researchers    researchers   
47  publications   resulting_publication_doi         string   
48  publications        supporting_grant_ids         string   
49  publications                 times_cited        integer   
50  publications                       title         string   
51  publications                        type         string   
52  publications                      volume         string   
53  publications                        year        integer   

                                          description  is_filter  is_entity  \
0                          Altmetric attention score.       True      False   
1                            AltMetric Publication ID       True      False   
2   Ordered list of authors names and their affili...       True      False   
3   The DOI of the book a chapter belongs to (note...       True      False   
4       The title of the book series book, belong to.      False      False   
5   The title of the book a chapter belongs to (no...      False     

### 3.1 `where`

This optional phrase consists of the keyword `where` followed by a
`filters` phrase consisting of DSL filter expressions, as described
below.

In [21]:
%dsldf search publications where type = "book" return publications limit 5

Returned Publications: 5 (total = 289033)


year  type                                              title  \
0  2020  book                  Scholia ad libros E - I continens   
1  2020  book                               Rap - Text - Analyse   
2  2020  book                Framing Power in Visigothic Society   
3  2020  book        Duoethnography in English Language Teaching   
4  2020  book  Heidi - Diversität in Kunst, Wissenschaft und ...   

               id  
0  pub.1108414342  
1  pub.1124907345  
2  pub.1125801672  
3  pub.1125300609  
4  pub.1124907285

If a `for` phrase is also used in a filtered query, the
system will first apply the filters, and then search the resulting
restricted set of documents for the `search term`.

In [22]:
%dsldf search publications for "malaria" where type = "book" return publications limit 5

Returned Publications: 5 (total = 12348)


year  type                                              title  \
0  2020  book                   Phagocytes and Cellular Immunity   
1  2020  book  Pharmaceutical Drug Product Development and Pr...   
2  2020  book         Routledge Handbook of Ecocultural Identity   
3  2020  book  The Routledge Companion to Italian Fascist Arc...   
4  2020  book                             Clinical Neurovirology   

               id  
0  pub.1127540316  
1  pub.1127312535  
2  pub.1127315933  
3  pub.1127251872  
4  pub.1127249846

### 3.2 `in`

For convenience, the DSL also supports shorthand notation for filters
where a particular field should be restricted to a specified range or
list of values (although the same logic may be expressed using complex
filters as shown below).

Syntax: a **range filter** consists of the `field` name, the keyword `in`, and a
range of values enclosed in square brackets (`[]`), where the range
consists of a `low` value, colon `:`, and a `high` value.

In [23]:
%%dsldf 
search grants 
    for "malaria" 
    where start_year in [ 2010 : 2015 ] 
return grants limit 5

Returned Grants: 5 (total = 3046)


id                                              title  \
0  grant.4729738  Bloodborne tropical pathogen detection using m...   
1  grant.4729736  Field-deployable Assay for Differential Diagno...   
2  grant.4729699  T cell driven antigen discovery for vaccine ca...   
3  grant.4854433  Senior Fellowship for Dr. Eduardo Samo Gudo: E...   
4  grant.8821176  Biology, Ecology & Management of Emerging Dise...   

     end_date title_language  project_num  \
0  2017-11-30             en  R21AI120981   
1  2019-02-28             en  R21AI120973   
2  2018-11-30             en  R21AI109439   
3  2018-12-18             en        91488   
4  2019-09-30             en   MIS-311250   

                                      original_title  \
0  Bloodborne tropical pathogen detection using m...   
1  Field-deployable Assay for Differential Diagno...   
2  T cell driven antigen discovery for vaccine ca...   
3  Senior Fellowship for Dr. Eduardo Samo Gudo: E...   
4  Biology, Ecology & Management of Emerging Dise...   

                                    funding_org_name language  \
0  National Institute of Allergy and Infectious D...       en   
1  National Institute of Allergy and Infectious D...       en   
2  National Institute of Allergy and Infectious D...       en   
3                              Volkswagen Foundation       en   
4         National Institute of Food and Agriculture       en   

                                             funders  start_date  \
0  [{'id': 'grid.419681.3', 'linkout': ['http://w...  2015-12-28   
1  [{'id': 'grid.419681.3', 'linkout': ['http://w...  2015-12-24   
2  [{'id': 'grid.419681.3', 'linkout': ['http://w...  2015-12-21   
3  [{'id': 'grid.452969.5', 'linkout': ['https://...  2015-12-18   
4  [{'id': 'grid.482914.2', 'linkout': ['https://...  2015-12-10   

                      active_year  start_year  
0              [2015, 2016, 2017]        2015  
1  [2015, 2016, 2017, 2018, 2019]        2015  
2        [2015, 2016, 2017, 2018]        2015  
3        [2015, 2016, 2017, 2018]        2015  
4  [2015, 2016, 2017, 2018, 2019]        2015

Syntax: a **list filter** consists of the `field` name, the keyword `in`, and a list
of one or more `value` s enclosed in square brackets (`[]`), where
values are separated by commas (`,`):

In [24]:
%%dsldf 
search grants 
    for "malaria" 
    where research_org_name in [ "UC Berkeley", "UC Davis", "UCLA"  ] 
return grants limit 5

Returned Grants: 0
WARNINGS [1]
Field 'research_org_name' is deprecated in favor of research_orgs. Please refer to https://docs.dimensions.ai/dsl/releasenotes.html for more details


Empty DataFrame
Columns: []
Index: []

### 3.3 `count` - filter function

The filter function `count` is supported on some fields in
[publications](publications.ipynb) (e.g. `researchers` and
`research_orgs`).

Use of this filter is shown on the example below:

In [25]:
%%dsldf 
search publications 
    for "malaria" 
    where count(research_orgs) > 5 
return research_orgs limit 5

Returned Research_orgs: 5


id  count   latitude    country_name  city_name        types  \
0    grid.4991.5   1473  51.753437  United Kingdom     Oxford  [Education]   
1    grid.8991.9   1393  51.520900  United Kingdom     London  [Education]   
2  grid.38142.3c   1013  42.377052   United States  Cambridge  [Education]   
3  grid.21107.35    813  39.328888   United States  Baltimore  [Education]   
4    grid.7445.2    729  51.498600  United Kingdom     London  [Education]   

                                           name  longitude  \
0                          University of Oxford  -1.254010   
1  London School of Hygiene & Tropical Medicine  -0.130700   
2                            Harvard University -71.116650   
3                      Johns Hopkins University -76.620280   
4                       Imperial College London  -0.175478   

                        linkout     state_name acronym  
0        [http://www.ox.ac.uk/]    Oxfordshire     NaN  
1     [http://www.lshtm.ac.uk/]         Camden   LSHTM  
2     [http://www.harvard.edu/]  Massachusetts     NaN  
3        [https://www.jhu.edu/]       Maryland     JHU  
4  [http://www.imperial.ac.uk/]    Westminster     NaN

Number of publications with more than 50 researcher.

In [26]:
%%dsldf 
search publications 
    for "malaria" 
    where count(researchers) > 50 
return publications limit 5

Returned Publications: 5 (total = 181)


id                                              title  \
0  pub.1127418736  Mapping geographical inequalities in childhood...   
1  pub.1126151286  Genetic tool development in marine protists: e...   
2  pub.1127157285  Frequency and management of maternal infection...   
3  pub.1127247220  A SARS-CoV-2 protein interaction map reveals t...   
4  pub.1125560167  Triple artemisinin-based combination therapies...   

                                 author_affiliations     type  year  \
0  [[{'first_name': 'Robert C', 'last_name': 'Rei...  article  2020   
1  [[{'first_name': 'Drahomíra', 'last_name': 'Fa...  article  2020   
2  [[{'first_name': 'Mercedes', 'last_name': 'Bon...  article  2020   
3  [[{'first_name': 'David E.', 'last_name': 'Gor...  article  2020   
4  [[{'first_name': 'Rob W', 'last_name': 'van de...  article  2020   

     journal.id             journal.title volume  issue      pages  
0  jour.1077219                The Lancet    NaN    NaN        NaN  
1  jour.1033763            Nature Methods     17      5    481-494  
2  jour.1048786  The Lancet Global Health      8      5  e661-e671  
3  jour.1018957                    Nature    NaN    NaN       1-13  
4  jour.1077219                The Lancet    395  10233  1345-1360

Number of publications with more than one researcher.

In [27]:
%%dsldf 
search publications
where count(researchers) > 1
return funders limit 5

Returned Funders: 5


id    count   latitude   country_name acronym  city_name  \
0  grid.419696.5  1750945  40.005177          China    NSFC    Beijing   
1  grid.270680.b   644018  50.851650        Belgium      EC   Brussels   
2  grid.424020.0   563135  39.827835          China    MOST    Beijing   
3  grid.48336.3a   553879  39.004326  United States     NCI  Rockville   
4  grid.54432.34   523939  35.687160          Japan    JSPS      Tokyo   

          types                                               name  \
0  [Government]       National Natural Science Foundation of China   
1  [Government]                                European Commission   
2  [Government]  Ministry of Science and Technology of the Peop...   
3  [Government]                          National Cancer Institute   
4   [Nonprofit]         Japan Society for the Promotion of Science   

    longitude                                    linkout state_name  
0  116.339830  [http://www.nsfc.gov.cn/publish/portal1/]        NaN  
1    4.363670         [http://ec.europa.eu/index_en.htm]        NaN  
2  116.316284              [http://www.most.gov.cn/eng/]        NaN  
3  -77.101190                   [http://www.cancer.gov/]   Maryland  
4  139.740390                   [http://www.jsps.go.jp/]        NaN

International collaborations: number of publications with more than one author and affiliations located in more than one country.

In [28]:
%%dsldf 
search publications
where count(researchers) > 1
and count(research_org_countries) > 1
return funders limit 5

Returned Funders: 5


id   count   latitude country_name acronym city_name  \
0  grid.419696.5  429900  40.005177        China    NSFC   Beijing   
1  grid.270680.b  329590  50.851650      Belgium      EC  Brussels   
2  grid.424150.6  149281  50.699340      Germany     DFG      Bonn   
3  grid.424020.0  142455  39.827835        China    MOST   Beijing   
4  grid.54432.34  131928  35.687160        Japan    JSPS     Tokyo   

          types                                               name  \
0  [Government]       National Natural Science Foundation of China   
1  [Government]                                European Commission   
2    [Facility]                         German Research Foundation   
3  [Government]  Ministry of Science and Technology of the Peop...   
4   [Nonprofit]         Japan Society for the Promotion of Science   

    longitude                                    linkout  
0  116.339830  [http://www.nsfc.gov.cn/publish/portal1/]  
1    4.363670         [http://ec.europa.eu/index_en.htm]  
2    7.147797                    [http://www.dfg.de/en/]  
3  116.316284              [http://www.most.gov.cn/eng/]  
4  139.740390                   [http://www.jsps.go.jp/]

Domestic collaborations: number of publications with more than one author and more than one affiliation located in exactly one country.

In [29]:
%%dsldf 
search publications
where count(researchers) > 1
and count(research_org_countries) = 1
return funders limit 5

Returned Funders: 5


id    count                                        linkout  \
0  grid.419696.5  1279844      [http://www.nsfc.gov.cn/publish/portal1/]   
1  grid.424020.0   409720                  [http://www.most.gov.cn/eng/]   
2  grid.48336.3a   405932                       [http://www.cancer.gov/]   
3  grid.54432.34   359781                       [http://www.jsps.go.jp/]   
4  grid.280785.0   313763  [http://www.nigms.nih.gov/Pages/default.aspx]   

                                                name   country_name  \
0       National Natural Science Foundation of China          China   
1  Ministry of Science and Technology of the Peop...          China   
2                          National Cancer Institute  United States   
3         Japan Society for the Promotion of Science          Japan   
4     National Institute of General Medical Sciences  United States   

          types   longitude  city_name   latitude acronym state_name  
0  [Government]  116.339830    Beijing  40.005177    NSFC        NaN  
1  [Government]  116.316284    Beijing  39.827835    MOST        NaN  
2  [Government]  -77.101190  Rockville  39.004326     NCI   Maryland  
3   [Nonprofit]  139.740390      Tokyo  35.687160    JSPS        NaN  
4    [Facility]  -77.099380   Bethesda  38.997833   NIGMS   Maryland

### 3.4 Filter Operators

A simple filter expression consists of a `field` name, an in-/equality
operator `op`, and the desired field `value`. 

The `value` must be a
`string` enclosed in double quotes (`"`) or an integer (e.g. `1234`).

The available operators are:

| `op`           | meaning                                                                                  |
|----------------|------------------------------------------------------------------------------------------|
| `=`            | *is* (or *contains* if the given `field` is multi-value)                                 |
| `!=`           | *is not*                                                                                 |
| `>`            | *is greater than*                                                                        |
| `<`            | *is less than*                                                                           |
| `>=`           | *is greater than or equal to*                                                            |
| `<=`           | *is less than or equal to*                                                               |
| `~`            | *partially matches* (see partial-string-matching below) |
| `is empty`     | *is empty* (see emptiness-filters below)                      |
| `is not empty` | *is not empty* (see emptiness-filters below)                  |

A couple of examples 

In [30]:
%dsldf search datasets where year > 2010 and year < 2012 return datasets limit 5

Returned Datasets: 5 (total = 38341)


year                                              title  \
0  2011  (Table 1) Radiocarbon ages of samples taken fr...   
1  2011  Average fluorescence and dissolved iron and Fe...   
2  2011  (Table 1) Average fluorescence in the surface ...   
3  2011  Dissolved and dissolvable iron concentrations ...   
4  2011  (Table 1) Movement parameters of nine adult fe...   

                                             authors        id   keywords  \
0  [{'name': 'Minna Väliranta', 'orcid': ''}, {'n...  10993892  [PANGAEA]   
1  [{'name': 'Charles-Edouard Thuróczy', 'orcid':...  10993247  [PANGAEA]   
2  [{'name': 'Charles-Edouard Thuróczy', 'orcid':...  10993244  [PANGAEA]   
3  [{'name': 'Charles-Edouard Thuróczy', 'orcid':...  10993241  [PANGAEA]   
4  [{'name': 'Jean-François Therrien', 'orcid': '...  10993193  [PANGAEA]   

     journal.id                                      journal.title  
0  jour.1020344                            Journal of Biogeography  
1  jour.1023157  Deep Sea Research Part II Topical Studies in O...  
2           NaN                                                NaN  
3  jour.1312079                    Journal of Geophysical Research  
4  jour.1023041                           Journal of Avian Biology

In [31]:
%dsldf search patents where assignees != "grid.410484.d" return patents limit 5

Returned Patents: 5 (total = 39704493)


assignees  year  \
0  [{'id': 'grid.6584.f', 'country_name': 'German...  2001   
1                                                NaN  2009   
2  [{'id': 'grid.418190.5', 'country_name': 'Unit...  1996   
3  [{'id': 'grid.471210.1', 'country_name': 'Japa...  1998   
4  [{'id': 'grid.471143.4', 'country_name': 'Japa...  1997   

                                               title publication_date  \
0  METHODS FOR OPERATING A MOTOR VEHICLE DRIVEN B...       2009-12-09   
1  TARGETED TELEVISION ADVERTISEMENTS ASSOCIATED ...       2009-12-10   
2  MULTI-CYCLE LOOP INJECTION FOR TRACE ANALYSIS ...       2009-12-09   
3                    Waste water treatment apparatus       2009-12-09   
4  Optical waveguide grating and production metho...       2009-12-09   

  filing_status                          assignee_names  times_cited  \
0         Grant  [Robert Bosch GmbH, BOSCH GMBH ROBERT]            0   
1   Application                            [SHKEDI ROY]            1   
2         Grant              [Dionex Corp, DIONEX CORP]            0   
3         Grant            [Kuraray Co Ltd, KURARAY CO]            1   
4         Grant            [Fujikura Ltd, FUJIKURA LTD]            0   

   granted_year                id  \
0        2009.0     EP-1409282-B1   
1           NaN  WO-2009149128-A2   
2        2009.0     EP-0868664-B1   
3        2009.0     EP-0861808-B1   
4        2009.0     EP-0805365-B1   

                                      inventor_names  
0           [TUMBACK, STEFAN, SCHNELLE, KLAUS-PETER]  
1                                      [SHKEDI, ROY]  
2               [RIVIELLO, JOHN, M., REY, MARIA, A.]  
3  [TANAKA, EIJI, HIGASHI, TAMIO, KITAMURA, TAKAN...  
4  [NAKAI, MICHIHIRO, SHIMA, KENSUKE, HIDAKA, HIR...

### 3.5 Partial string matching with `~`

The `~` operator indicates that the given `field` need only partially,
instead of exactly, match the given `string` (the `value` used with this
operator must be a `string`, not an integer).

For example, the filter `where research_orgs.name~"Saarland Uni"` would
match both the organization named "Saarland University" and the one
named "Universitätsklinikum des Saarlandes", and any other organization
whose name includes the terms "Saarland" and "Uni" (the order is
unimportant). 

In [32]:
%%dsldf 
search patents 
    where assignee_names ~ "IBM" 
return assignees limit 5

Returned Assignees: 5


id   count    country_name      city_name  \
0  grid.410484.d  329418   United States         Armonk   
1  grid.471366.1   22089  Cayman Islands    George Town   
2  grid.14648.3f    5071  United Kingdom     Winchester   
3  grid.420451.6    3555   United States  Mountain View   
4  grid.472772.3    2717           China        Beijing   

                               name  
0               IBM (United States)  
1  GlobalFoundries (Cayman Islands)  
2              IBM (United Kingdom)  
3                            Google  
4                    Lenovo (China)

### 3.6 Emptiness filters `is empty`

To filter records which contain specific field or to filter those which
contain an empty field, it is possible to use something like
`where research_orgs is not empty` or `where issn is empty`.

In [33]:
%%dsldf
search publications 
    for "iron graphene" 
    where researchers is empty 
    and research_orgs is not empty 
return publications[id+title+researchers+research_orgs+type] limit 5

Returned Publications: 5 (total = 2342)


id                                              title  \
0  pub.1126814951  Highly Selective Detection of Metronidazole by...   
1  pub.1127580003  Microstructure Change of Nanosilica–Cement Com...   
2  pub.1127607284  Suppression of superconductivity and enhanced ...   
3  pub.1127615869  Improved Cycling Stability of LiCoO2 at 4.5 V ...   
4  pub.1125095130  Molecular Dynamics Simulations of Melting Iron...   

                                       research_orgs     type  
0  [{'id': 'grid.459947.2', 'linkout': ['http://w...  article  
1  [{'id': 'grid.449845.0', 'linkout': ['http://w...  article  
2  [{'id': 'grid.7340.0', 'linkout': ['http://www...  article  
3  [{'id': 'grid.54549.39', 'linkout': ['http://e...  article  
4  [{'id': 'grid.33764.35', 'linkout': ['http://e...  article

## 4. Searching for Researchers

The DSL offers different mechanisms for searching for researchers (e.g.
publication authors, grant investigators), each of them presenting
specific advantages.

### 4.1 Exact name searches

Special full-text indices allows to look up a researcher's name and
surname **exactly as they appear in the source documents** they derive from.

This approach has a broad scope, as it allows to search the full
collection of Dimensions documents irrespectively of whether a
researcher was succesfully disambiguated (and hence given a Dimensions
ID). On the other hand, this approach will only match names as they
appear in the source document, so different spellings or initials are
not necessarily returned via a single query. 

```
search in [authors|investigators|inventors]
```

It is possible to look up publications authors using a specific
`search index` called `authors`. 

This method expects case insensitive
phrases, in format $"<first name> <last name>"$ or reverse order. Note
that strings in double quotes that contain nested quotes must always be
escaped by a backslash `\`.

In [34]:
%dsldf search publications in authors for "\"Charles Peirce\"" return publications limit 5

Returned Publications: 5 (total = 196)


title  \
0                         WHAT IS A SIGN?   
1                     Evolutionary Love 1   
2          The Probability of Induction 1   
3               The Doctrine of Chances 1   
4  Deduction, Induction, and Hypothesis 1   

                                 author_affiliations    pages     type  year  \
0  [[{'first_name': 'Charles', 'last_name': 'Peir...  400-408  chapter  2019   
1  [[{'first_name': 'Charles S', 'last_name': 'Pe...  267-300  chapter  2017   
2  [[{'first_name': 'Charles S', 'last_name': 'Pe...   82-105  chapter  2017   
3  [[{'first_name': 'Charles S', 'last_name': 'Pe...    61-81  chapter  2017   
4  [[{'first_name': 'Charles S', 'last_name': 'Pe...  131-153  chapter  2017   

               id  
0  pub.1127675133  
1  pub.1110025901  
2  pub.1110025905  
3  pub.1110025904  
4  pub.1110025907

Instead of first name, initials can also be used. These are examples of
valid research search phrases:

-   `\"Peirce, Charles S.\"`
-   `\"Charles S. Peirce\"`
-   `\"CS Peirce\"`
-   `\"Peirce CS\"`
-   `\"C S Peirce\"`
-   `\"Peirce C S\"`
-   `\"C Peirce\"`
-   `\"Peirce C\"`
-   `\"Charles Peirce\"`
-   `\"Peirce Charles\"`

**Warning**: In order to produce valid results an author or an investigator search
query must contain **at least two components or more** (e.g., name and
surname, either in full or initials).

Investigators search is similar to *authors* search, only it allows to search on `grants` and
`clinical trials` using a separate search index `investigators`, and on
`patents` using the index `inventors`.

In [35]:
%%dsldf 
search clinical_trials in investigators for "\"John Smith\"" 
return clinical_trials limit 5

Returned Clinical_trials: 2 (total = 2)


title           id  \
0  VEPTR Implantation to Treat Children With Earl...  NCT00689533   
1  Prospective Evaluation of Symptom Resolution i...  NCT01241149   

                                        active_years  \
0  [2008, 2009, 2010, 2011, 2012, 2013, 2014, 201...   
1                                                NaN   

                                investigator_details  
0  [[John M Flynn, MD, Principal Investigator, Ch...  
1  [[Ellie Mentler, MD, Principal Investigator, U...

In [36]:
%%dsldf 
search grants in investigators for "\"Satoko Shimazaki\"" 
return grants limit 5

Returned Grants: 4 (total = 4)


language                                     original_title  \
0       en  Kabuki Actors, Print Technology, and the Theat...   
1       ja           古・中英語期における女性聖人伝の系譜研究：Aelfricのテクストと言語を中心に   
2       en  Images of Women in the Old English Lives of Sa...   
3       en  Reception and Transfromation of the Images of ...   

                                               title title_language  \
0  Kabuki Actors, Print Technology, and the Theat...             en   
1  Genealogy research on female saints in the Pal...             ja   
2  Images of Women in the Old English Lives of Sa...             en   
3  Reception and Transfromation of the Images of ...             en   

                                             funders  \
0  [{'id': 'grid.422239.c', 'latitude': 38.884052...   
1  [{'id': 'grid.54432.34', 'latitude': 35.68716,...   
2  [{'id': 'grid.54432.34', 'latitude': 35.68716,...   
3  [{'id': 'grid.54432.34', 'latitude': 35.68716,...   

                active_year  start_year  \
0              [2020, 2021]        2020   
1  [2018, 2019, 2020, 2021]        2018   
2  [2015, 2016, 2017, 2018]        2015   
3  [2012, 2013, 2014, 2015]        2012   

                             funding_org_name    project_num             id  \
0       National Endowment for the Humanities  FEL-263245-19  grant.7925589   
1  Japan Society for the Promotion of Science       18K00431  grant.7527261   
2  Japan Society for the Promotion of Science       15K02313  grant.5858713   
3  Japan Society for the Promotion of Science       24520310  grant.6086985   

   start_date    end_date  
0  2020-09-01  2021-08-31  
1  2018-04-01  2021-03-31  
2  2015-04-01  2018-03-31  
3  2012-04-01  2015-03-31

In [37]:
%%dsldf 
search patents in inventors for "\"John Smith\"" 
return patents limit 5

Returned Patents: 5 (total = 501)


title  \
0                                  Diagnostic method   
1                          Automotive heat exchanger   
2  Microelectronic assemblies with composite cond...   
3  A lockable safety insert for an electrical dom...   
4                               Ammunition cartridge   

                                      assignee_names  times_cited  \
0                 [AstraZeneca AB, SMITH JOHN CRAIG]            0   
1  [Llanelli Radiators Ltd, Calsonic Kansei UK Lt...            0   
2                         [Tessera Inc, TESSERA INC]            2   
3                                       [SMITH JOHN]            0   
4                               [Eley Ltd, ELEY LTD]            0   

                                           assignees inventor_names  \
0  [{'id': 'grid.418151.8', 'country_name': 'Swed...   [John Smith]   
1  [{'id': 'grid.472810.8', 'country_name': 'Unit...   [SMITH JOHN]   
2  [{'id': 'grid.455499.0', 'country_name': 'Unit...   [John Smith]   
3                                                NaN   [SMITH JOHN]   
4                                                NaN   [SMITH JOHN]   

  filing_status publication_date  year                 id  granted_year  
0   Application       2002-10-31  2001  US-20020160362-A1           NaN  
1         Grant       2006-03-22  2002       GB-2384299-B        2006.0  
2   Application       2005-06-23  2005  US-20050133900-A1           NaN  
3         Grant       2004-11-03  2003    IE-S20030195-A2        2004.0  
4   Application       2014-10-22  2013       GB-2513101-A           NaN

### 4.2 Fuzzy Searches

This type of search is similar to *full-text
search*, with the difference that it
allows searching by only a part of a name, e.g. only the 'last name' of
a person, by using the `where` clause. 

**Note** At this moment, this type of search is only available for
`publications`. Other sources will add this option in the future.

For example:

In [38]:
%%dsldf 
search publications where authors = "Hawking" 
return publications limit 5[id+doi+title+authors] limit 10

Returned Errors: 1
1 QuerySyntaxError found
1 ParserError found
  * [Line 2:27] ('[') mismatched input '[' expecting <EOF>


Generally speaking, using a `where` clause to search authors is less
precise that using the relevant exact-search syntax. 

On the other hand, using a
`where` clause can be handy if one wants to **combine an author search
with another full-text search index**.

For example:

In [39]:
%%dsldf 
search publications 
    in title_abstract_only for "dna replication" 
    where authors = "smith"  
return publications limit 5

Returned Publications: 5 (total = 1521)


id                                              title volume  \
0  pub.1124910780  Genetic associations with clozapine-induced my...     10   
1  pub.1125664041  An epigenome-wide association study of posttra...     12   
2  pub.1124060243  Longitudinal epigenome-wide association studie...     12   
3  pub.1126387158  Molecular Targeting of Cancer-Associated PCNA ...     17   
4  pub.1125466205  Reciprocal monoallelic expression of ASAR lncR...     26   

                                 author_affiliations     type  year issue  \
0  [[{'first_name': 'Paul', 'last_name': 'Lacaze'...  article  2020     1   
1  [[{'first_name': 'Mark W.', 'last_name': 'Logu...  article  2020     1   
2  [[{'first_name': 'Clara', 'last_name': 'Snijde...  article  2020     1   
3  [[{'first_name': 'Shanna J.', 'last_name': 'Sm...  article  2020   NaN   
4  [[{'first_name': 'Michael', 'last_name': 'Hesk...  article  2020     6   

            pages    journal.id                   journal.title  
0              37  jour.1045271        Translational Psychiatry  
1              46  jour.1042271            Clinical Epigenetics  
2              11  jour.1042271            Clinical Epigenetics  
3         250-256  jour.1052368  Molecular Therapy - Oncolytics  
4  rna.073114.119  jour.1114285                             RNA

### 4.3 Using the disambiguated Researchers database

The Dimensions [Researchers](https://docs.dimensions.ai/dsl/datasource-researchers.html) source is a database of
researchers information algorithmically extracted and disambiguated from
all of the other content sources (publications, grants, clinical trials
etc..).

By using the `researchers` source it is possible to match an
'aggregated' person object linking together multiple publication
authors, grant investigators etc.. irrespectively of the form their
names can take in the original source documents.

However, since database does not contain all authors and investigators information
available in Dimensions. 

E.g. think of authors from older publications,
or authors with very common names that are difficult to disambiguate, or
very new authors, who have only one or few publications. In such cases,
using full-text authors search might be more
appropriate.

Examples:

In [40]:
%%dsldf 
search researchers for "\"Satoko Shimazaki\"" 
return researchers[basics+obsolete] 

Returned Researchers: 4 (total = 4)


id  last_name first_name  \
0  ur.014307627665.09  Shimazaki     Satoko   
1   ur.07751146721.59  Shimazaki     Satoko   
2  ur.010537333602.30  Shimazaki     Satoko   
3  ur.015527473602.63  Shimazaki     Satoko   

                                       research_orgs  obsolete  
0  [{'id': 'grid.19006.3e', 'linkout': ['http://w...         0  
1                                                NaN         0  
2                                                NaN         1  
3  [{'id': 'grid.266190.a', 'linkout': ['http://w...         0

NOTE pay attentiont to the `obsolete` field. This indicates the researcher ID status. 0 means that the researcher ID is still **active**, 1 means that the researcher ID is **no longer valid**. This is due to the ongoing process of refinement of Dimensions researchers. 

Hence the query above is best written like this:

In [41]:
%%dsldf 
search researchers where obsolete=0 for "\"Satoko Shimazaki\"" 
return researchers[basics+obsolete] 

Returned Researchers: 3 (total = 3)


first_name  last_name                  id  obsolete  \
0     Satoko  Shimazaki  ur.014307627665.09         0   
1     Satoko  Shimazaki   ur.07751146721.59         0   
2     Satoko  Shimazaki  ur.015527473602.63         0   

                                       research_orgs  
0  [{'id': 'grid.19006.3e', 'latitude': 34.072224...  
1                                                NaN  
2  [{'id': 'grid.266190.a', 'latitude': 40.007366...

With `Researchers`, one can use other fields as well:

In [42]:
%%dsldf 
search researchers 
    where obsolete=0 and last_name="Shimazaki" 
return researchers[basics] limit 5

Returned Researchers: 5 (total = 470)


first_name  last_name                  id  \
0     Tatsuo  Shimazaki  ur.013510032403.65   
1     Tomomi  Shimazaki  ur.010700310627.87   
2        Dai  Shimazaki  ur.011035131473.19   
3       Koji  Shimazaki  ur.016627632300.80   
4     Kuniko  Shimazaki   ur.01266043223.28   

                                       research_orgs  
0  [{'id': 'grid.419075.e', 'latitude': 37.41523,...  
1  [{'id': 'grid.471199.3', 'latitude': 34.923733...  
2  [{'id': 'grid.415776.6', 'latitude': 35.774574...  
3                                                NaN  
4  [{'id': 'grid.272456.0', 'latitude': 35.66406,...

## 5. Returning results

After the `search` phrase, a query must contain one or more `return`
phrases, specifying the content and format of the information that
should be returned.



### 5.1 Returning Multiple Sources

Multiple results may not be returned in a single `return` phrase.

In [43]:
%%dsldf 
search publications 
return funders limit 5 
return research_orgs limit 5 
return year

Returned Year: 20
Returned Funders: 5
Returned Research_orgs: 5
[Warning] Dataframe created from first available key, but more than one JSON key found: ['year', 'funders', 'research_orgs']


id    count
0   2019  5475372
1   2018  5115784
2   2017  4785031
3   2016  4400992
4   2015  4218909
5   2014  4077119
6   2013  3882407
7   2012  3624257
8   2011  3505652
9   2010  3083192
10  2009  2953124
11  2007  2804422
12  2008  2780841
13  2006  2488664
14  2020  2314009
15  2005  2282625
16  2004  2163570
17  2003  1972215
18  2002  1841441
19  2001  1779485


### 5.2 Returning Specific Fields

For control over which information from each given `record` will be
returned, a `source` or `entity` name in the `results` phrase can be
optionally followed by a specification of `fields` and `fieldsets` to be
included in the JSON results for each retrieved record.

The fields specification may be an arbitrary list of `field` names
enclosed in brackets (`[`, `]`), with field names separated by a plus
sign (`+`). Minus sign (`-`) can be used to exclude `field` or a
`fieldset` from the result. Field names thus listed within brackets must
be "known" to the DSL, and therefore only a subset of fields may be used
in this syntax (see note below).

In [44]:
%%dsldf 
search grants 
return grants[grant_number + title + language] limit 5

Returned Grants: 5 (total = 5310256)


language                                              title    grant_number
0       en  APPROACH to Enriching the Real World Evidence ...  2018-HRSI-1548
1       en  Molecular mechanism of DNA double strand break...        1301720F
2       en                     Life as concept and as science          M 2734
3       en  Scintillation Light For New Physics with Liqui...          892933
4       en  Jet quenching for heavy-ion collisions at the LHC          893021

In [45]:
%%dsldf 
search clinical_trials 
return clinical_trials [id+ title + acronym + phase] limit 5

Returned Clinical_trials: 5 (total = 560033)


id                                              title    phase  \
0  NCT00249756     Re-Entry MTC for Offenders With MICA Disorders      NaN   
1  NCT00249782  A Phase II, Randomized, Partial-Blind, Paralle...  Phase 2   
2  NCT00249795  A Parallel Randomized Controlled Evaluation of...  Phase 3   
3  NCT00249847  A Feasibility Study of Positron Emission Tomog...      NaN   
4  NCT00249860  A Multicentre Phase III Study of Interferon-be...  Phase 3   

    acronym  
0       NaN  
1       NaN  
2  ACTIVE I  
3       NaN  
4       NaN

**Shortcuts: `fieldsets`**

The fields specification may be the name of a pre-defined `fieldset`
(e.g. `extras`, `basics`). These are shortcuts that can be handy when testing out new queries, for example. 

NOTE In general when writing code used in integrations or long-standing extraction scripts it is **best to return specific fields rather that a predefined set**. This has also the advantage of making queries faster by avoiding the extraction of unnecessary data.
    

In [46]:
%%dsldf 
search grants 
return grants [basics] limit 5 

Returned Grants: 5 (total = 5310256)
WARNINGS [2]
Field 'title_language' is deprecated in favor of language_title. Please refer to https://docs.dimensions.ai/dsl/releasenotes.html for more details
Field 'project_num' is deprecated in favor of grant_number. Please refer to https://docs.dimensions.ai/dsl/releasenotes.html for more details


title title_language  \
0  APPROACH to Enriching the Real World Evidence ...             en   
1  Molecular mechanism of DNA double strand break...             en   
2                     Life as concept and as science             en   
3  Scintillation Light For New Physics with Liqui...             en   
4  Jet quenching for heavy-ion collisions at the LHC             en   

   start_year                                            funders  start_date  \
0        2021  [{'id': 'grid.484521.e', 'country_name': 'Cana...  2021-11-30   
1        2021  [{'id': 'grid.424470.1', 'country_name': 'Belg...  2021-10-01   
2        2021  [{'id': 'grid.25111.36', 'country_name': 'Aust...  2021-10-01   
3        2021  [{'id': 'grid.270680.b', 'country_name': 'Belg...  2021-09-01   
4        2021  [{'id': 'grid.270680.b', 'country_name': 'Belg...  2021-09-01   

                           funding_org_name         active_year language  \
0  New Brunswick Health Research Foundation              [2021]       en   
1              Fund for Scientific Research              [2021]       en   
2                 FWF Austrian Science Fund  [2021, 2022, 2023]       en   
3                       European Commission  [2021, 2022, 2023]       en   
4                       European Commission  [2021, 2022, 2023]       en   

      project_num                                     original_title  \
0  2018-HRSI-1548  APPROACH to Enriching the Real World Evidence ...   
1        1301720F  Mécanismes moléculaires de la formation et la ...   
2          M 2734                     Life as concept and as science   
3          892933  Scintillation Light For New Physics with Liqui...   
4          893021  Jet quenching for heavy-ion collisions at the LHC   

              id    end_date  
0  grant.8690978         NaN  
1  grant.8950252         NaN  
2  grant.8715161  2023-09-30  
3  grant.8964235  2023-08-31  
4  grant.8963889  2023-08-31

In [47]:
%%dsldf 
search publications 
return publications [basics+times_cited] limit 5 

Returned Publications: 5 (total = 109848472)
WARNINGS [1]
Field 'author_affiliations' is deprecated in favor of authors. Please refer to https://docs.dimensions.ai/dsl/releasenotes.html for more details


id                                              title  \
0  pub.1125931386  Visual research on the trustability of classic...   
1  pub.1125801610  5. ‘Martyrs of Love’. Genesis, Development and...   
2  pub.1125788851  Introduction: Murra, Materialism, Anthropology...   
3  pub.1125789246  22. Structure and application of the slanting ...   
4  pub.1125801609                               4. Perpetual Contest   

                                 author_affiliations     type  year    pages  \
0  [[{'first_name': 'Nihal', 'last_name': 'ATA TU...  article  2020     1-18   
1  [[{'first_name': 'Asghar', 'last_name': 'Seyed...  chapter  2020  129-152   
2  [[{'first_name': 'John S.', 'last_name': 'Hend...  chapter  2020      1-8   
3  [[{'first_name': 'Masahiko', 'last_name': 'Nis...  chapter  2020  313-318   
4  [[{'first_name': 'Mieke', 'last_name': 'Bal', ...  chapter  2020  105-128   

   times_cited    journal.id                                    journal.title  
0            0  jour.1142190  Hacettepe Journal of Mathematics and Statistics  
1            0           NaN                                              NaN  
2            0           NaN                                              NaN  
3            0           NaN                                              NaN  
4            0           NaN                                              NaN

The fields specification may be an (`all`), to indicate that all fields
available for the given `source` should be returned.

In [48]:
%%dsldf
search publications 
return publications [all] limit 5 

Returned Publications: 5 (total = 109848482)
WARNINGS [10]
Field 'FOR' is deprecated in favor of category_for. Please refer to https://docs.dimensions.ai/dsl/releasenotes.html for more details
Field 'author_affiliations' is deprecated in favor of authors. Please refer to https://docs.dimensions.ai/dsl/releasenotes.html for more details
Field 'terms' is deprecated in favor of concepts. Please refer to https://docs.dimensions.ai/dsl/releasenotes.html for more details
Field 'RCDC' is deprecated in favor of category_rcdc. Please refer to https://docs.dimensions.ai/dsl/releasenotes.html for more details
Field 'open_access' is deprecated in favor of open_access_categories. Please refer to https://docs.dimensions.ai/dsl/releasenotes.html for more details
Field 'references' is deprecated in favor of reference_ids. Please refer to https://docs.dimensions.ai/dsl/releasenotes.html for more details
Field 'HRCS_RAC' is deprecated in favor of category_hrcs_rac. Please refer to https://docs.dimension

doi  \
0      10.15672/hujms.630402   
1  10.1515/9789048540211-008   
2  10.7591/9781501734977-002   
3  10.7591/9781501737688-031   
4  10.1515/9789048540211-007   

                              open_access_categories        date  \
0  [{'id': 'oa_all', 'description': 'Article is f...  2020-12-31   
1  [{'id': 'closed', 'description': 'No freely av...  2020-12-31   
2  [{'id': 'closed', 'description': 'No freely av...  2020-12-31   
3  [{'id': 'closed', 'description': 'No freely av...  2020-12-31   
4  [{'id': 'closed', 'description': 'No freely av...  2020-12-31   

  date_inserted  recent_citations     type  year                    issn  \
0    2020-03-28                 0  article  2020  [2651-477X, 1300-4263]   
1    2020-03-22                 0  chapter  2020                     NaN   
2    2020-03-22                 0  chapter  2020                     NaN   
3    2020-03-22                 0  chapter  2020                     NaN   
4    2020-03-22                 0  chapter  2020                     NaN   

   altmetric_id                                           concepts  ...  \
0             0                       [Cox regression, regression]  ...   
1             0  [martyrs, love, genesis, development, politica...  ...   
2             0                                                NaN  ...   
3             0                                                NaN  ...   
4             0                                                NaN  ...   

                                         researchers times_cited  \
0  [{'id': 'ur.015425340575.47', 'first_name': 'N...           0   
1  [{'id': 'ur.015312661111.06', 'first_name': 'A...           0   
2  [{'id': 'ur.013500506607.34', 'first_name': 'P...           0   
3                                                NaN           0   
4                                                NaN           0   

                                         publisher  \
0  Hacettepe Journal of Mathematics and Statistics   
1                                       De Gruyter   
2                         Cornell University Press   
3                         Cornell University Press   
4                                       De Gruyter   

                                               title    pages              id  \
0  Visual research on the trustability of classic...     1-18  pub.1125931386   
1  5. ‘Martyrs of Love’. Genesis, Development and...  129-152  pub.1125801610   
2  Introduction: Murra, Materialism, Anthropology...      1-8  pub.1125788851   
3  22. Structure and application of the slanting ...  313-318  pub.1125789246   
4                               4. Perpetual Contest  105-128  pub.1125801609   

     journal.id                                    journal.title  \
0  jour.1142190  Hacettepe Journal of Mathematics and Statistics   
1           NaN                                              NaN   
2           NaN                                              NaN   
3           NaN                                              NaN   
4           NaN                                              NaN   

                 book_title               book_doi  
0                       NaN                    NaN  
1                 Martyrdom  10.1515/9789048540211  
2   Configurations of Power  10.7591/9781501734977  
3  Problem Snake Management  10.7591/9781501737688  
4                 Martyrdom  10.1515/9789048540211  

[5 rows x 29 columns]

### 5.3 Returning Facets

In addition to returning source records matching a query, it is possible
to $facet$ on the [entity](data-entities.ipynb) fields related to a
particular source and return only those entity values as an aggregrated
view of the related source data. This operation is similar to a
$group by$ or $pivot table$.

**Warning** Faceting can return up to a maximum of 1000 results. This is to ensure
adequate performance with all queries. Furthemore, although the `limit`
operator is allowed, the `skip` operator cannot be used.

In [49]:
%%dsldf 
search publications 
    for "coronavirus" 
return research_orgs limit 5

Returned Research_orgs: 5


id  count                    linkout  \
0  grid.194645.b    961       [http://www.hku.hk/]   
1  grid.21107.35    794     [https://www.jhu.edu/]   
2  grid.38142.3c    710  [http://www.harvard.edu/]   
3  grid.25879.31    699    [http://www.upenn.edu/]   
4  grid.17063.33    654  [http://www.utoronto.ca/]   

                         name   country_name        types  longitude  \
0     University of Hong Kong          China  [Education]  114.13708   
1    Johns Hopkins University  United States  [Education]  -76.62028   
2          Harvard University  United States  [Education]  -71.11665   
3  University of Pennsylvania  United States  [Education]  -75.19322   
4       University of Toronto         Canada  [Education]  -79.39500   

      city_name   latitude acronym     state_name  
0     Hong Kong  22.283287     HKU      Hong Kong  
1     Baltimore  39.328888     JHU       Maryland  
2     Cambridge  42.377052     NaN  Massachusetts  
3  Philadelphia  39.952457     NaN   Pennsylvania  
4       Toronto  43.661667     NaN        Ontario

In [50]:
%%dsldf 
search publications 
    for "coronavirus" 
return research_org_countries limit 5
return year limit 5
return category_for limit 5

Returned Year: 5
Returned Research_org_countries: 5
Returned Category_for: 5
[Warning] Dataframe created from first available key, but more than one JSON key found: ['year', 'research_org_countries', 'category_for']


id  count
0  2020  29022
1  2008   6905
2  2018   5614
3  2011   4808
4  2016   4513

For control over the organization and headers of the JSON query results,
the `return` keyword in a return phrase may be followed by the keyword
`in` and then a `group` name for this group of results, where the group
name is enclosed in double quotes(`"`).

Also, one can define `aliases` that replace the defaul JSON fields names with other ones provided by the user. 

See the [official documentation](https://docs.dimensions.ai/dsl/language.html#aliases) for more details about this feature. 

In [51]:
%%dsldf 
search publications 
return in "facets" funders 
return in "facets" research_orgs

Returned Facets: 2


funders  \
0  [{'id': 'grid.419696.5', 'count': 1943482, 'la...   

                                       research_orgs  
0  [{'id': 'grid.26999.3d', 'count': 324888, 'lat...

### 5.4 What the query statistics refer to - sources VS facets

When performing a DSL search, a `_stats` object is return which contains some useful info eg the total number of records available for a search. 

In [52]:
%%dsldf 
search publications
  where year in [2013:2018] and research_orgs="grid.258806.1"
return publications limit 5

Returned Publications: 5 (total = 3769)


title  \
0  Development of Organo-Dispersible Graphene Oxi...   
1  Nuclear Ab Initio Calculations with the Unitar...   
2  Anisotropic Crystal Growth, Optical Absorption...   
3  Indium Zinc Oxide Electron Transport Layer for...   
4  Chalcopyrite ZnSnSb2: A Promising Thermoelectr...   

                                 author_affiliations volume issue  \
0  [[{'first_name': 'Siewteng', 'last_name': 'Sim...      3    12   
1  [[{'first_name': 'T.', 'last_name': 'Miyagi', ...    NaN   NaN   
2  [[{'first_name': 'Taro', 'last_name': 'Toyoda'...    122    51   
3  [[{'first_name': 'Liang', 'last_name': 'Wang',...    122    50   
4  [[{'first_name': 'Ami', 'last_name': 'Nomura',...     10    50   

         pages        type  year              id    journal.id  \
0  18124-18131     article  2018  pub.1110885950  jour.1157000   
1          NaN  proceeding  2018  pub.1110925389           NaN   
2  29200-29209     article  2018  pub.1110369527  jour.1038386   
3  28491-28496     article  2018  pub.1110271601  jour.1038386   
4  43682-43690     article  2018  pub.1110222625  jour.1041450   

                         journal.title  
0                            ACS Omega  
1                                  NaN  
2  The Journal of Physical Chemistry C  
3  The Journal of Physical Chemistry C  
4   ACS Applied Materials & Interfaces



It is important to note though that the **total number always refers to the main source, never the facets** one is searching for. 

For example, in this query we return `researchers` linked to publications: 

In [53]:
%%dsldf 
search publications
  where year in [2013:2018] and research_orgs="grid.258806.1"
return researchers limit 5

Returned Researchers: 5


id  count last_name   first_name  \
0   ur.01055753603.27    138    Hayase  Shuzi Shuzi   
1  ur.011212042763.67    102    Hikita     Masayuki   
2   ur.01144540527.52     98        Ma      Ting-Li   
3   ur.07644453127.11     96    Kozako   M Kozako M   
4  ur.016357156077.09     91        Lu       Huimin   

                                       research_orgs               orcid_id  
0  [grid.14003.36, grid.258806.1, grid.266298.1, ...                    NaN  
1      [grid.462727.2, grid.27476.30, grid.258806.1]                    NaN  
2  [grid.11135.37, grid.443434.0, grid.411485.d, ...  [0000-0002-3310-459X]  
3  [grid.11417.32, grid.258806.1, grid.5290.e, gr...                    NaN  
4  [grid.16821.3c, grid.1024.7, grid.454850.8, gr...  [0000-0001-9794-3221]

NOTE: facet results can be 1000 at most (due to performance limitations) so if there are more than 1000 it is not possible to know the total number. 

### 5.5 Paginating Results

At the end of a `return` phrase, the user can specify the maximum number
of results to be returned and the number of top records to skip over
before returning the first result record, for e.g. returning large
result sets page-by-page (i.e. "paging" results) as described below.

This is done using the keyword `limit` followed by the maximum number of
results to return, optionally followed by the keyword `skip` and the
number of results to skip (the offset).

In [54]:
%%dsldf 
search publications return publications limit 10

Returned Publications: 10 (total = 109848482)


title  \
0  Visual research on the trustability of classic...   
1  5. ‘Martyrs of Love’. Genesis, Development and...   
2  Introduction: Murra, Materialism, Anthropology...   
3  22. Structure and application of the slanting ...   
4                               4. Perpetual Contest   
5                   2. The Sunflowers in Perspective   
6                                       Introduction   
7                              Sojourners in the Sun   
8               Dilthey and the Narrative of History   
9                     Musical Meaning and Expression   

                                 author_affiliations    pages       type  \
0  [[{'first_name': 'Nihal', 'last_name': 'ATA TU...     1-18    article   
1  [[{'first_name': 'Asghar', 'last_name': 'Seyed...  129-152    chapter   
2  [[{'first_name': 'John S.', 'last_name': 'Hend...      1-8    chapter   
3  [[{'first_name': 'Masahiko', 'last_name': 'Nis...  313-318    chapter   
4  [[{'first_name': 'Mieke', 'last_name': 'Bal', ...  105-128    chapter   
5  [[{'first_name': 'Nienke', 'last_name': 'Bakke...    21-48    chapter   
6  [[{'first_name': 'Jan Willem', 'last_name': 'v...    11-32    chapter   
7  [[{'first_name': 'Alan', 'last_name': 'Karras'...      NaN  monograph   
8  [[{'first_name': 'Jacob', 'last_name': 'Owensb...      NaN  monograph   
9  [[{'first_name': 'Stephen', 'last_name': 'Davi...      NaN  monograph   

   year              id    journal.id  \
0  2020  pub.1125931386  jour.1142190   
1  2020  pub.1125801610           NaN   
2  2020  pub.1125788851           NaN   
3  2020  pub.1125789246           NaN   
4  2020  pub.1125801609           NaN   
5  2020  pub.1125801740           NaN   
6  2020  pub.1125801605           NaN   
7  2020  pub.1125788975           NaN   
8  2020  pub.1125789145           NaN   
9  2020  pub.1125788819           NaN   

                                     journal.title  
0  Hacettepe Journal of Mathematics and Statistics  
1                                              NaN  
2                                              NaN  
3                                              NaN  
4                                              NaN  
5                                              NaN  
6                                              NaN  
7                                              NaN  
8                                              NaN  
9                                              NaN

If paging information is not provided, the default values
`limit 20 skip 0` are used, so the two following queries are equivalent:

Combining `limit` and `skip` across multiple queries enables paging or
batching of results; e.g. to retrieve 30 grant records divided into 3
pages of 10 records each, the following three queries could be used:

```
return grants limit 10           => get 1st 10 records for page 1 (skip 0, by default)
return grants limit 10 skip 10   => get next 10 for page 2; skip the 10 we already have
return grants limit 10 skip 20   => get another 10 for page 3, for a total of 30
```

### 5.6 Sorting Results

A sort order for the results in a given `return` phrase can be specified
with the keyword `sort by` followed by the name of 
* a `field` (in the
case that a `source` is being requested) 
* an `indicator (aggregation)` (in the case
that one or more facets are being requested). 

 By default, the result set of full text
queries ($search ... for "full text query"$) is sorted by "relevance".
Additionally, it is possible to specify the sort order, using `asc` or
`desc` keywords. By default, descending order is selected.

In [55]:
%%dsldf 
search grants 
    for "nanomaterials"
return grants sort by title desc limit 5 

Returned Grants: 5 (total = 17719)


language                    original_title  \
0       pl                                 x   
1       en  Transmissionselektronenmikroskop   
2       de  Transmissionselektronenmikroskop   
3       en                      Snowcontrol.   
4       de                     Röntgenquelle   

                              title title_language  \
0                                 x             pl   
1  Transmissionselektronenmikroskop             en   
2  Transmissionselektronenmikroskop             en   
3                      Snowcontrol.             en   
4                     Röntgenquelle             en   

                                             funders  \
0  [{'id': 'grid.436846.b', 'latitude': 50.073513...   
1  [{'id': 'grid.424150.6', 'latitude': 50.69934,...   
2  [{'id': 'grid.424150.6', 'latitude': 50.69934,...   
3  [{'id': 'grid.425119.a', 'latitude': 50.82542,...   
4  [{'id': 'grid.424150.6', 'latitude': 50.69934,...   

                      active_year  start_year  \
0        [2019, 2020, 2021, 2022]        2019   
1                          [2015]        2015   
2                          [2012]        2012   
3  [2011, 2012, 2013, 2014, 2015]        2011   
4                          [2014]        2014   

                        funding_org_name          project_num             id  \
0                National Science Center  2018/29/N/ST5/01240  grant.8518592   
1             German Research Foundation            280331443  grant.4841519   
2             German Research Foundation            220923099  grant.4823271   
3  Belgian Federal Science Policy Office             3E120109  grant.6774902   
4             German Research Foundation            245513494  grant.4834305   

   start_date    end_date  
0  2019-04-01  2022-03-31  
1  2015-01-01         NaN  
2  2012-01-01         NaN  
3  2011-06-16  2015-06-13  
4  2014-01-01         NaN

In [56]:
%%dsldf  
search grants  
    for "nanomaterials"
return grants  sort by relevance desc limit 5

Returned Grants: 5 (total = 17719)


language                         original_title  \
0       en  Optically-active chiral nanomaterials   
1       en                  Polymer Nanomaterials   
2       en                  Polymer Nanomaterials   
3       en                  Polymer Nanomaterials   
4       en                  Polymer Nanomaterials   

                                   title title_language  \
0  Optically-active chiral nanomaterials             en   
1                  Polymer Nanomaterials             en   
2                  Polymer Nanomaterials             en   
3                  Polymer Nanomaterials             en   
4                  Polymer Nanomaterials             en   

                                             funders   active_year  \
0  [{'id': 'grid.437854.9', 'latitude': 53.333664...  [2012, 2013]   
1  [{'id': 'grid.452912.9', 'latitude': 45.425594...  [2016, 2017]   
2  [{'id': 'grid.452912.9', 'latitude': 45.425594...  [2016, 2017]   
3  [{'id': 'grid.452912.9', 'latitude': 45.425594...  [2013, 2014]   
4  [{'id': 'grid.452912.9', 'latitude': 45.425594...  [2010, 2011]   

   start_year                                   funding_org_name  \
0        2012                         Science Foundation Ireland   
1        2016  Natural Sciences and Engineering Research Council   
2        2016  Natural Sciences and Engineering Research Council   
3        2013  Natural Sciences and Engineering Research Council   
4        2010  Natural Sciences and Engineering Research Council   

    project_num             id  start_date    end_date  
0  11/W.1/I2065  grant.3984032  2012-06-01  2013-05-31  
1        617505  grant.6973622  2016-04-01  2017-03-31  
2        617153  grant.6973270  2016-04-01  2017-03-31  
3        543663  grant.3643972  2013-04-01  2014-03-31  
4        454382  grant.2865162  2010-04-01  2011-03-31

Number of citations per publication

In [57]:
%%dsldf  
search publications
return publications  [doi + times_cited] 
    sort by times_cited limit 5

Returned Publications: 5 (total = 109848482)


times_cited                           doi
0       230709                           NaN
1       196626              10.1038/227680a0
2       178451  10.1016/0003-2697(76)90527-3
3        87041        10.1006/meth.2001.1262
4        82575   10.1103/physrevlett.77.3865

Recent citations per publication.
Note: Recent citation refers to the number of citations accrued in the last two year period. A single value is stored per document and the year window rolls over in July.

In [58]:
%%dsldf 
search publications
return publications [doi + recent_citations]
    sort by recent_citations limit 5

Returned Publications: 5 (total = 109848472)


recent_citations                               doi
0             28974            10.1006/meth.2001.1262
1             21685       10.1103/physrevlett.77.3865
2             21030  10.1176/appi.books.9780890425596
3             20464              10.1109/cvpr.2016.90
4             19629         10.1191/1478088706qp063oa

When a facet is being returned, the `indicator` used in the
`sort` phrase must either be `count` (the default, such that
`sort by count` is unnecessary), or one of the indicators specified in
the `aggregate` phrase, i.e. one whose values are being computed in the
faceting operation. 


In [59]:
%%dsldf 
search publications 
    for "nanomaterials"
return research_orgs 
    aggregate altmetric_median, rcr_avg sort by rcr_avg limit 5 

Returned Research_orgs: 5


id  count     rcr_avg  altmetric_median  \
0  grid.11444.34      1  207.839996             343.0   
1  grid.11485.39      1  207.839996             343.0   
2  grid.11642.30      1  207.839996             343.0   
3  grid.120073.7      1  207.839996             343.0   
4  grid.20931.39      1  207.839996             343.0   

                                             linkout  \
0                        [http://www.china-sih.com/]   
1                 [http://www.cancerresearchuk.org/]   
2  [http://www.univ-reunion.fr/university-of-reun...   
3      [http://www.cuh.org.uk/addenbrookes-hospital]   
4                            [http://www.rvc.ac.uk/]   

                                 name    country_name         types  \
0  Shanghai Institute of Hypertension           China    [Facility]   
1                  Cancer Research UK  United Kingdom   [Nonprofit]   
2            University of La Réunion         Reunion   [Education]   
3              Addenbrooke's Hospital  United Kingdom  [Healthcare]   
4            Royal Veterinary College  United Kingdom   [Education]   

    longitude    city_name   latitude acronym      state_name  
0  121.467255     Shanghai  31.211678     NaN             NaN  
1   -0.106269       London  51.531322    CRUK             NaN  
2   55.484550  Saint-Denis -20.901735     NaN             NaN  
3    0.140000    Cambridge  52.176000     NaN  Cambridgeshire  
4   -0.134000       London  51.536800     RVC             NaN

## 6. Aggregations

In a `return` phrase requesting one or more `facet` results, aggregation
operations to perform during faceting can be specified after the facet
name(s) by using the keyword `aggregate` followed by a comma-separated
list of one or more `indicator` names corresponding to the `source`
being searched.

In [60]:
%%dsldf
search publications 
    where year > 2010 
return research_orgs  
    aggregate rcr_avg, altmetric_median limit 5

Returned Research_orgs: 5


id   count   rcr_avg  altmetric_median    country_name  \
0  grid.17063.33  140357  1.692509          3.757726          Canada   
1  grid.38142.3c  135955  2.212846          5.000000   United States   
2  grid.11899.38  131793  1.045939          2.000000          Brazil   
3  grid.83440.3b  120256  1.906585          4.000000  United Kingdom   
4  grid.26999.3d  118760  1.181061          2.000000           Japan   

      state_name                         linkout        types  city_name  \
0        Ontario       [http://www.utoronto.ca/]  [Education]    Toronto   
1  Massachusetts       [http://www.harvard.edu/]  [Education]  Cambridge   
2            NaN        [http://www5.usp.br/en/]  [Education]  São Paulo   
3            NaN         [http://www.ucl.ac.uk/]  [Education]     London   
4            NaN  [http://www.u-tokyo.ac.jp/en/]  [Education]      Tokyo   

    latitude                       name   longitude acronym  
0  43.661667      University of Toronto  -79.395000     NaN  
1  42.377052         Harvard University  -71.116650     NaN  
2 -23.563051    University of São Paulo  -46.730103     USP  
3  51.524470  University College London   -0.133982     UCL  
4  35.713333        University of Tokyo  139.762220      UT

**What are the metrics/aggregations available?** See the data sources documentation for information about available [indicators](https://docs.dimensions.ai/dsl/datasource-publications.html#publications-indicators).  

Alternatively, we can use the 'schema' API ([describe](https://docs.dimensions.ai/dsl/data-sources.html#metadata-api)) to return this information programmatically:

In [61]:
schema = dsl.query("describe schema")
sources = [x for x in schema['sources']]
# for each source name, extract metrics info
for s in sources:
    print("SOURCE:", s)
    for m in schema['sources'][s]['metrics']:
        print("--", schema['sources'][s]['metrics'][m]['name'], " => ", schema['sources'][s]['metrics'][m]['description'], )

SOURCE: publications
-- count  =>  Total count
-- altmetric_median  =>  Median Altmetric attention score
-- altmetric_avg  =>  Altmetric attention score mean
-- citations_total  =>  Aggregated number of citations
-- citations_avg  =>  Arithmetic mean of citations
-- citations_median  =>  Median of citations
-- recent_citations_total  =>  For a given article, in a given year, the number of citations accrued in the last two year period. Single value stored per document, year window rolls over in July.
-- rcr_avg  =>  Arithmetic mean of `relative_citation_ratio` field.
-- fcr_gavg  =>  Geometric mean of `field_citation_ratio` field (note: This field cannot be used for sorting results).
SOURCE: grants
-- count  =>  Total count
-- funding  =>  Total funding amount, in USD.
SOURCE: patents
-- count  =>  Total count
SOURCE: clinical_trials
-- count  =>  Total count
SOURCE: policy_documents
-- count  =>  Total count
SOURCE: researchers
-- count  =>  Total count
SOURCE: organizations
-- count  

**NOTE** In addition to any specified aggregations, `count` is always computed
and reported when facet results are requested.

In [62]:
%%dsldf
search grants 
    for "5g network" 
return funders 
    aggregate count, funding sort by funding limit 5 

Returned Funders: 5


id  count      funding  \
0  grid.270680.b    175  834354500.0   
1  grid.421091.f     68   52650403.0   
2  grid.457785.c    106   49446108.0   
3  grid.55047.33      5   47182381.0   
4  grid.457810.f     73   24371660.0   

                                       linkout  \
0           [http://ec.europa.eu/index_en.htm]   
1                   [https://www.epsrc.ac.uk/]   
2  [http://www.nsf.gov/dir/index.jsp?org=CISE]   
3                 [http://www.ncbr.gov.pl/en/]   
4   [http://www.nsf.gov/dir/index.jsp?org=ENG]   

                                                name    country_name  \
0                                European Commission         Belgium   
1  Engineering and Physical Sciences Research Cou...  United Kingdom   
2  Directorate for Computer & Information Science...   United States   
3       National Centre for Research and Development          Poland   
4                        Directorate for Engineering   United States   

          types  longitude  city_name   latitude   acronym state_name  
0  [Government]   4.363670   Brussels  50.851650        EC        NaN  
1  [Government]  -1.784602    Swindon  51.567093     EPSRC    England  
2  [Government] -77.111000  Arlington  38.880580  NSF CISE   Virginia  
3  [Government]  21.007630     Warsaw  52.227455      NCRD        NaN  
4  [Government] -77.111000  Arlington  38.880580   NSF ENG   Virginia

Aggregated total number of citations

In [63]:
%%dsldf
search publications
    for "ontologies"
return funders 
    aggregate citations_total 
    sort by citations_total  limit 5

Returned Funders: 5


id  count  citations_total    country_name state_name  \
0  grid.48336.3a  12056         800359.0   United States   Maryland   
1  grid.280785.0  11565         771208.0   United States   Maryland   
2  grid.280128.1   4419         571771.0   United States   Maryland   
3  grid.270680.b  17977         544168.0         Belgium        NaN   
4  grid.52788.30   4823         415876.0  United Kingdom        NaN   

                                         linkout         types  city_name  \
0                       [http://www.cancer.gov/]  [Government]  Rockville   
1  [http://www.nigms.nih.gov/Pages/default.aspx]    [Facility]   Bethesda   
2                      [https://www.genome.gov/]    [Facility]   Bethesda   
3             [http://ec.europa.eu/index_en.htm]  [Government]   Brussels   
4                   [http://www.wellcome.ac.uk/]   [Nonprofit]     London   

    latitude acronym                                            name  \
0  39.004326     NCI                       National Cancer Institute   
1  38.997833   NIGMS  National Institute of General Medical Sciences   
2  38.996967   NHGRI        National Human Genome Research Institute   
3  50.851650      EC                             European Commission   
4  51.525867      WT                                  Wellcome Trust   

   longitude  
0 -77.101190  
1 -77.099380  
2 -77.096930  
3   4.363670  
4  -0.135005

Arithmetic mean number of citations

In [64]:
%%dsldf
search publications
return funders 
    aggregate citations_avg 
    sort by citations_avg limit 5

Returned Funders: 5


id  count  citations_avg   country_name            state_name  \
0  grid.478308.0    168     276.732143  United States  District of Columbia   
1  grid.453780.d    143     186.174825  United States  District of Columbia   
2  grid.478789.d    568     164.378521  United States                Nevada   
3  grid.417710.4    181     161.646409  United States              Maryland   
4  grid.429197.0    719     146.186370  United States              New York   

                           linkout        types        city_name  latitude  \
0  [http://www.stewart-trust.org/]  [Nonprofit]  Washington D.C.  38.90116   
1           [http://www.abc2.org/]  [Nonprofit]  Washington D.C.  38.90672   
2     [http://www.dwreynolds.org/]      [Other]        Las Vegas  36.19046   
3            [http://www.hgsi.com]    [Company]        Rockville  39.09665   
4           [http://www.hhwf.org/]      [Other]         New City  41.15845   

                                    name   longitude  
0     Alexander & Margaret Stewart Trust  -77.039730  
1           Accelerate Brain Cancer Cure  -77.039520  
2          Donald W. Reynolds Foundation -115.299850  
3  Human Genome Sciences (United States)  -77.203760  
4           Helen Hay Whitney Foundation  -73.982895

Geometric mean of FCR


In [65]:
%%dsldf
search publications
return funders 
    aggregate fcr_gavg limit 5

Returned Funders: 5


id  fcr_gavg    count   country_name  \
0  grid.419696.5  2.299626  1943482          China   
1  grid.270680.b  3.270630   676365        Belgium   
2  grid.424020.0  2.516387   610297          China   
3  grid.48336.3a  4.879969   583711  United States   
4  grid.54432.34  2.250702   573627          Japan   

                                     linkout         types  city_name  \
0  [http://www.nsfc.gov.cn/publish/portal1/]  [Government]    Beijing   
1         [http://ec.europa.eu/index_en.htm]  [Government]   Brussels   
2              [http://www.most.gov.cn/eng/]  [Government]    Beijing   
3                   [http://www.cancer.gov/]  [Government]  Rockville   
4                   [http://www.jsps.go.jp/]   [Nonprofit]      Tokyo   

    latitude acronym                                               name  \
0  40.005177    NSFC       National Natural Science Foundation of China   
1  50.851650      EC                                European Commission   
2  39.827835    MOST  Ministry of Science and Technology of the Peop...   
3  39.004326     NCI                          National Cancer Institute   
4  35.687160    JSPS         Japan Society for the Promotion of Science   

    longitude state_name  
0  116.339830        NaN  
1    4.363670        NaN  
2  116.316284        NaN  
3  -77.101190   Maryland  
4  139.740390        NaN

Median Altmetric Attention Score

In [66]:
%%dsldf 
search publications
return funders aggregate altmetric_median 
    sort by altmetric_median limit 5 

Returned Funders: 5


id  count  altmetric_median    country_name  \
0  grid.258806.1      6             309.0           Japan   
1  grid.470711.4      2             110.5  United Kingdom   
2  grid.443873.f      5              99.0   United States   
3  grid.473856.b      2              66.0   United States   
4  grid.473769.8      1              33.0   United States   

                                linkout         types        city_name  \
0  [https://www.kyutech.ac.jp/english/]   [Education]       Kitakyushu   
1             [http://www.chss.org.uk/]   [Nonprofit]        Edinburgh   
2           [http://www.lungevity.org/]   [Nonprofit]          Chicago   
3            [https://www.acf.hhs.gov/]  [Government]  Washington D.C.   
4                [http://www.bcan.org/]   [Nonprofit]         Bethesda   

    latitude acronym                                      name   longitude  \
0  33.894436     KIT            Kyushu Institute of Technology  130.839200   
1  55.946075    CHSS           Chest Heart and Stroke Scotland   -3.219597   
2  41.878674    LUNG                      LUNGevity Foundation  -87.626480   
3  38.885940     ACF  Administration for Children and Families  -77.016370   
4  38.988724    BCAN           Bladder Cancer Advocacy Network  -77.097880   

             state_name  
0                   NaN  
1                   NaN  
2              Illinois  
3  District of Columbia  
4              Maryland